In [14]:
# Importar bibliotecas

import pandas as pd
import pulp

In [15]:
# Crear/leer datos

def leer_datos() -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.Timestamp]:
    """Crea DataFrames con los datos de los proyectos, periodos y muelles; y la fecha inicial.

    Returns
    -------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.
    fecha_inicial : str
        Fecha inicial del primer periodo de los proyectos en formato 'YYYY-MM-DD'. 
    """    

    # Aquí lógica para leer los datos desde un archivo o base de datos
    proyectos = pd.DataFrame({
        'eslora': [120, 100, 120],
        'manga': [18, 15, 18],
        'proyecto_id': ['PRO1', 'PRO2', 'PRO3'],
        'nombre_area': ['sin_ubicacion_asignada', 'sin_ubicacion_asignada', 'SUR'],
        'proyecto_a_optimizar': [True, True, False]})
    
    proyectos.set_index('proyecto_id', inplace=True)

    periodos = pd.DataFrame({
        'tipo_desc': ['FLOTE', 'FLOTE', 'FLOTE'],
        'fecha_inicio': ['2025-08-08', '2025-08-10', '2025-08-17'],
        'fecha_fin': ['2025-08-20', '2025-08-16', '2025-08-25'],
        'proyecto_id': ['PRO1', 'PRO2', 'PRO3'],
        'periodo_id': [0, 0, 0]})
    
    periodos['fecha_inicio'] = pd.to_datetime(periodos['fecha_inicio'])
    periodos['fecha_fin'] = pd.to_datetime(periodos['fecha_fin'])
    periodos['id_proyecto_reparacion'] = periodos['proyecto_id'] + '_' + periodos['periodo_id'].astype(str)
    periodos.set_index('id_proyecto_reparacion', inplace=True)

    muelles = pd.DataFrame({
        'longitud': [130, 110],
        'ancho': [20, 20],
        'nombre': ['SUR', 'NORTE']})

    muelles.set_index('nombre', inplace=True)

    fecha_inicial = periodos['fecha_inicio'].min()

    return proyectos, periodos, muelles, fecha_inicial


In [52]:
# Preprocesar datos

def preprocesar_datos(proyectos: pd.DataFrame, periodos: pd.DataFrame, muelles: pd.DataFrame, fecha_inicial: pd.Timestamp) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, list, set, set, dict]:
    """Preprocesa los datos de proyectos, periodos y muelles para su uso en la optimización.

    Parameters
    ----------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame  
        DataFrame con las dimensiones de los muelles.
    fecha_inicial : str
        Fecha inicial del primer periodo de los proyectos en formato 'YYYY-MM-DD'.
    
    Returns
    -------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos, con fechas convertidas a enteros.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.
    dias : list
        Lista de días desde la fecha inicial hasta la fecha final de los periodos.
    """

    # Convertir fechas a integer
    periodos['fecha_inicio'] = (periodos['fecha_inicio'] - fecha_inicial).dt.days
    periodos['fecha_fin'] = (periodos['fecha_fin'] - fecha_inicial).dt.days

    # Crear una lista de días
    dias = list(range(periodos['fecha_inicio'].min(), periodos['fecha_fin'].max()+1))

    # Crear set de proyectos confirmados y sin confirmar
    set_sinConfirmar = set(proyectos[proyectos['proyecto_a_optimizar']].index)
    set_confirmados = set(proyectos[~proyectos['proyecto_a_optimizar']].index)

    # Crear diccionario de longitud total de barcos confirmados por ubicación
    longitudes_confirmados = proyectos[~proyectos['proyecto_a_optimizar']].groupby('nombre_area')['eslora'].sum().to_dict()

    return proyectos, periodos, muelles, dias, set_confirmados, set_sinConfirmar, longitudes_confirmados


In [53]:
# Definir variables de decisión

def definir_variables(proyectos: pd.DataFrame, periodos: pd.DataFrame, muelles: pd.DataFrame, set_sinConfirmar: set) -> tuple[dict, dict, dict, dict]:
    """Define las variables de decisión del problema de optimización.

    Parameters
    ----------
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.

    Returns
    -------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    y : dict
        Diccionario de variables binarias que indican si un periodo está asignado (1) o no (0).
    dias_vars : dict
        Diccionario que mapea cada periodo a una lista de días correspondientes.
    locs_vars : dict
        Diccionario que mapea cada periodo a una lista de localizaciones disponibles para ese periodo.
    """
    
    # Definir variables para cada periodo solo en los días y localizaciones correspondientes

    periodos['locs'] = periodos.apply(lambda row: [m for m in muelles.index if (muelles.loc[m, 'longitud'] >= proyectos.loc[row['proyecto_id'], 'eslora'] and
                              muelles.loc[m, 'ancho'] >= proyectos.loc[row['proyecto_id'], 'manga'])], axis=1)
    periodos['dias'] = periodos.apply(lambda row: list(range(row['fecha_inicio'], row['fecha_fin'] + 1)), axis=1)

    locs_vars = periodos['locs'].to_dict()
    dias_vars = periodos['dias'].to_dict()

    x = {(p, d, loc): pulp.LpVariable(f"x_{p}_{d}_{loc}",(p, d, loc), cat='Binary')
         for p in periodos[periodos["proyecto_id"].isin(set_sinConfirmar)].index
         for d in dias_vars[p]
         for loc in locs_vars[p]}
    
    y = {p: pulp.LpVariable(f"y_{p}", p, cat='Binary')
         for p in set_sinConfirmar}
    
    return x, y, dias_vars, locs_vars

In [54]:
# Definir la función objetivo

def definir_funcion_objetivo(x: dict) -> pulp.LpAffineExpression:
    """Define la función objetivo del problema de optimización.

    Parameters
    ----------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.

    Returns
    -------
    objetivo : LpAffineExpression
        Expresión lineal que representa la función objetivo a maximizar.
    """

    objetivo = pulp.lpSum(x.values())
    return objetivo

In [61]:
# Definir restricciones

def definir_restricciones(x: dict, y: dict, dias: list, dias_vars: dict, locs_vars: dict, periodos: pd.DataFrame, muelles: pd.DataFrame, proyectos: pd.DataFrame, longitudes_confirmados: dict) -> dict:
    """Define las restricciones del problema de optimización.

    Parameters
    ----------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    y : dict
        Diccionario de variables binarias que indican si un periodo está asignado (1) o no (0).
    dias : list
        Lista de días desde la fecha inicial hasta la fecha final de los periodos.
    dias_vars : dict
        Diccionario que mapea cada periodo a una lista de días correspondientes.
    locs_vars : dict
        Diccionario que mapea cada periodo a una lista de localizaciones disponibles para ese periodo.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    muelles : pd.DataFrame
        DataFrame con las dimensiones de los muelles.
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    
    Returns
    -------
    restricciones : dict
        Diccionario de restricciones del problema de optimización.
    """

    restricciones = {}

    # Cada día del periodo debe estar asignado exactamente a un muelle si y[p] = 1 y a ninguno si y[p] = 0
    restricciones.update(
        {
            f"Asignacion_{p_k}_{d}": (pulp.lpSum(x[(p_k, d, loc)] for loc in locs_vars[p_k]) == y[p], f"Asignacion{p_k}_{d}")
            for p in proyectos[proyectos['proyecto_a_optimizar']].index
            for p_k in periodos[periodos["proyecto_id"] == p].index
            for d in dias_vars[p_k]
        }
    )

    # Los barcos en el mismo muelle no pueden exceder la longitud del muelle
    restricciones.update(
        {
            f"Longitud_Muelle_{d}_{loc}": (pulp.lpSum(x.get((p, d, loc),0) * proyectos.loc[periodos.loc[p, 'proyecto_id'], 'eslora'] for p in periodos.index) + longitudes_confirmados.get(loc,0) <= muelles.loc[loc, 'longitud'], 
            f"Longitud_Muelle_{d}_{loc}")
            for loc in muelles.index
            for d in dias
        }
    )

    return restricciones

In [62]:
# Resolver el problema de optimización

def resolver_problema(objetivo: pulp.LpAffineExpression, restricciones: dict) -> pulp.LpProblem:
    """Resuelve el problema de optimización utilizando PuLP.

    Parameters
    ----------
    objetivo : LpAffineExpression
        Expresión lineal que representa la función objetivo a maximizar.
    restricciones : dict
        Diccionario de restricciones del problema de optimización.
   
    Returns
    -------
    prob : LpProblem
        Objeto LpProblem que representa el problema de optimización.
    """

    prob = pulp.LpProblem("Asignación de Periodos a Muelles", pulp.LpMaximize)
    prob += objetivo

    for c in restricciones.values():
            prob += c

    prob.solve()

    return prob

In [57]:
# Imprimir asignación de proyectos por muelle

def imprimir_asignacion(prob, x, dias, periodos, muelles):
    
    print("Estado de la solución:", pulp.LpStatus[prob.status])
    print("\nAsignación de Proyectos a Muelles:\n")
    print("Día\t", "\t".join(muelles.index))

    for d in dias:
        row = f"{d}\t"
        for m in muelles.index:
            for p in periodos.index:
                if (p,d,m) in x.keys():
                    if x[(p,d,m)].varValue == 1:
                        row += f"{p}\t"
                        break
        print(row)


In [74]:
# Dataframe de resultados

def crear_dataframe_resultados(x: dict, dias_vars: dict, locs_vars: dict, proyectos: pd.DataFrame, periodos: pd.DataFrame, set_sinConfirmar: set, set_confirmados: set) -> pd.DataFrame:
    """Crea un DataFrame con los resultados de la asignación de periodos a muelles.

    Parameters
    ----------
    x : dict
        Diccionario de variables binarias que indican si un periodo está asignado a un muelle en un día específico.
    dias_vars : dict
        Diccionario que mapea cada periodo a una lista de días correspondientes.
    locs_vars : dict
        Diccionario que mapea cada periodo a una lista de localizaciones disponibles para ese periodo.
    proyectos : pd.DataFrame
        DataFrame con las dimensiones de los proyectos.
    periodos : pd.DataFrame
        DataFrame con los periodos de los proyectos.
    set_sinConfirmar : set
        Conjunto de proyectos sin confirmar.
    set_confirmados : set
        Conjunto de proyectos confirmados.
    
    Returns
    -------
    resultados : pd.DataFrame
        DataFrame con la asignación de periodos a muelles, incluyendo proyecto_id, periodo_id, ubicación, fecha_inicio y fecha_fin.
    """
    
    data = {
        'proyecto_id': [],
        'periodo_id': [],
        'ubicación': [],
        'fecha_inicio': [],
        'fecha_fin': []}
    
    for p in periodos[periodos["proyecto_id"].isin(set_sinConfirmar)].index:
        for loc in locs_vars[p]:
            if x[(p, dias_vars[p][0], loc)].varValue == 1:
                # Proyectos asignados por optimizador
                data['proyecto_id'].append(periodos.loc[p, 'proyecto_id'])
                data['periodo_id'].append(periodos.loc[p, 'periodo_id'])
                data['ubicación'].append(loc)
                data['fecha_inicio'].append(pd.to_datetime(periodos.loc[p, 'fecha_inicio'], unit='D', origin='2025-08-08'))
                data['fecha_fin'].append(pd.to_datetime(periodos.loc[p, 'fecha_fin'], unit='D', origin='2025-08-08'))
                break
        
        # Proyectos sin asignación por optimizador
        if periodos.loc[p, 'proyecto_id'] not in data['proyecto_id']:
            data['proyecto_id'].append(periodos.loc[p, 'proyecto_id'])
            data['periodo_id'].append(periodos.loc[p, 'periodo_id'])
            data['ubicación'].append(proyectos.loc[periodos.loc[p, 'proyecto_id'], 'nombre_area'])
            data['fecha_inicio'].append(pd.to_datetime(periodos.loc[p, 'fecha_inicio'], unit='D', origin='2025-08-08'))
            data['fecha_fin'].append(pd.to_datetime(periodos.loc[p, 'fecha_fin'], unit='D', origin='2025-08-08'))
    
    # Proyectos confirmados
    for p in periodos[periodos["proyecto_id"].isin(set_confirmados)].index:
        data['proyecto_id'].append(periodos.loc[p, 'proyecto_id'])
        data['periodo_id'].append(periodos.loc[p, 'periodo_id'])
        data['ubicación'].append(proyectos.loc[periodos.loc[p, 'proyecto_id'], 'nombre_area'])
        data['fecha_inicio'].append(pd.to_datetime(periodos.loc[p, 'fecha_inicio'], unit='D', origin='2025-08-08'))
        data['fecha_fin'].append(pd.to_datetime(periodos.loc[p, 'fecha_fin'], unit='D', origin='2025-08-08'))

    resultados = pd.DataFrame(data)
    resultados.index = resultados['proyecto_id'] + '_' + resultados['periodo_id'].astype(str)

    return resultados


In [75]:
# Optimize

def optimize():
    proyectos, periodos, muelles, fecha_inicial = leer_datos()
    proyectos, periodos, muelles, dias, set_confirmados, set_sinConfirmar, longitudes_confirmados = preprocesar_datos(proyectos, periodos, muelles, fecha_inicial)
    
    x, y, dias_vars, locs_vars = definir_variables(proyectos, periodos, muelles, set_sinConfirmar)
    objetivo = definir_funcion_objetivo(x)
    restricciones = definir_restricciones(x, y, dias, dias_vars, locs_vars, periodos, muelles, proyectos, longitudes_confirmados)
    
    prob = resolver_problema(objetivo, restricciones)
    resultados = crear_dataframe_resultados(x, dias_vars, locs_vars, proyectos, periodos, set_sinConfirmar, set_confirmados)
    print("Status:", pulp.LpStatus[prob.status])
    print("Resultados:\n\n", resultados)

In [76]:
optimize()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/77eeba1e0ee14a78a2b630088cd5588f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/x4/4ffrr56n3jzb55xn5fzcplwc0000gn/T/77eeba1e0ee14a78a2b630088cd5588f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 61 COLUMNS
At line 221 RHS
At line 278 BOUNDS
At line 308 ENDATA
Problem MODEL has 56 rows, 29 columns and 74 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 8.08333 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -7 to -1.79769e+308
Probing was tried 0 times and created 

/opt/miniconda3/envs/NAME_ENV/lib/python3.10/site-packages/pulp/pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
